<a href="https://colab.research.google.com/github/mdsaifin/BestOutOfWaste/blob/main/robonox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Project Name: Electronic Component detection using Machine learning Algorithm
# Developed by: MD SAIF
# For the Competition named "Best out of Waste" of Robonox Club at NIT Srinagar
# Dated: 19th to 25th March 2023
# Dataset link: https://drive.google.com/drive/folders/1cHvF_RfXRIOXUojGf6p_hNagjx1OwEwc?usp=sharing

In [2]:
import os
import numpy as np
from keras.applications import VGG16
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
!mkdir resistor_images command
!mkdir capacitor_images command
!mkdir diode_images command
!mkdir transistor_images command
!mkdir IC_images command

mkdir: cannot create directory ‘command’: File exists
mkdir: cannot create directory ‘command’: File exists
mkdir: cannot create directory ‘command’: File exists
mkdir: cannot create directory ‘command’: File exists


In [4]:
# Path to directory containing the component images
data_path_resistor = "resistor_images"
data_path_capacitor = "capacitor_images"
data_path_diode = "diode_images"
data_path_transistor = "transistor_images"
data_path_IC = "IC_images"

In [5]:
# Load resistor images and labels
resistor_images = []
resistor_labels = []
for filename in os.listdir(data_path_resistor):
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(data_path_resistor, filename))
        img = img.resize((224, 224))
        resistor_images.append(np.array(img))
        resistor_labels.append("resistor")

In [6]:
# Load capacitor images and labels
capacitor_images = []
capacitor_labels = []
for filename in os.listdir(data_path_capacitor):
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(data_path_capacitor, filename))
        img = img.resize((224, 224))
        capacitor_images.append(np.array(img))
        capacitor_labels.append("capacitor")

In [7]:
# Load diode images and labels
diode_images = []
diode_labels = []
for filename in os.listdir(data_path_diode):
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(data_path_diode, filename))
        img = img.resize((224, 224))
        diode_images.append(np.array(img))
        diode_labels.append("diode")

In [8]:
# Load transistor images and labels
transistor_images = []
transistor_labels = []
for filename in os.listdir(data_path_transistor):
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(data_path_transistor, filename))
        img = img.resize((224, 224))
        transistor_images.append(np.array(img))
        transistor_labels.append("transistor")

In [9]:
# Load IC images and labels
IC_images = []
IC_labels = []
for filename in os.listdir(data_path_IC):
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(data_path_IC, filename))
        img = img.resize((224, 224))
        IC_images.append(np.array(img))
        IC_labels.append("IC")

In [10]:
# Combine images and labels
images = np.concatenate((resistor_images, capacitor_images, diode_images, transistor_images, IC_images))
labels = np.concatenate((resistor_labels, capacitor_labels, diode_labels, transistor_labels, IC_labels))

In [11]:
# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

In [12]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

In [13]:
# Load pre-trained VGG16 model
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [14]:
# Extract features from images using VGG16
X_train = vgg.predict(np.array(X_train))
X_test = vgg.predict(np.array(X_test))
n_samples_train = X_train.shape[0]
n_samples_test = X_test.shape[0]
X_train = X_train.reshape(n_samples_train, -1)
X_test = X_test.reshape(n_samples_test, -1)

2/2 [==============================] - 32s 14s/step


In [15]:
# Train SVM model
svc = SVC(kernel='linear', C=1, probability=True)
svc.fit(X_train, y_train)

SVC(C=1, kernel='linear', probability=True)

In [16]:
# Evaluate model
y_pred = svc.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       0.97      1.00      0.98        28
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00        12
           4       1.00      0.83      0.91         6

    accuracy                           0.98        55
   macro avg       0.99      0.97      0.98        55
weighted avg       0.98      0.98      0.98        55

[[ 7  0  0  0  0]
 [ 0 28  0  0  0]
 [ 0  0  2  0  0]
 [ 0  0  0 12  0]
 [ 0  1  0  0  5]]


In [17]:
# Load image to test model
test_image = Image.open("test_r.jpg")
test_image = test_image.resize((224, 224))
test_image = np.array(test_image)
test_image = vgg.predict(np.array([test_image]))
n_samples_test = test_image.shape[0]
test_image = test_image.reshape(n_samples_test, -1)

1/1 [==============================] - 1s 997ms/step


In [18]:
# Predict label of test image
pred_label = le.inverse_transform(svc.predict(test_image))[0]

In [19]:
# Output result
if pred_label == "resistor":
    print("The component is resistor")
elif pred_label == "capacitor":
    print("The component is capacitor")
elif pred_label == "diode":
    print("The component is diode")
elif pred_label == "transistor":
    print("The component is transistor")
elif pred_label == "IC":
    print("The component is IC")    
else:
    print("Unknown component")

The component is resistor
